In [ ]:
# Set up interactive plotting using matplotlib, and load numpy
# %pylab ipympl
%pylab inline
import warnings
warnings.filterwarnings('ignore')

# 0: Introduction
In this notebook we will perform 3 common tasks: 

1. Obtain SMAP soil moisture data for a user-specified grid over a single date
2. Obtain SMAP soil moisture data for user-specified points over a range of dates
3. Obtain SMAP soil moisture data in its native coordinates
4. Export a pipeline for re-use elsewhere

# 00: Setup
## 00.0: Import PODPAC dependencies

In [ ]:
import podpac

## 00.1: Provide Earth Data Login Credentials
If you do not have an earth data login, follow the [instructions here](https://creare-com.github.io/podpac-docs/user/earthdata.html)

In [ ]:
import getpass
username = password = None
username = input("Username:");   password = getpass.getpass('Password:')

## 00.2: Create the PODPAC SMAP Node

In [ ]:
# Create the SMAP node
product = 'SPL4SMAU'   # Level 4 soil moisture analysis update
sm = podpac.datalib.SMAP(product=product, interpolation='nearest', username=username, password=password)

# 1: Retrieve and plot SMAP data for:
## * A particular date
## * Over lat-lon range with user-specified grid
# 1.0: Using PODPAC

In [ ]:
# dim = (start, stop, step)
lat =   (   90,  -90,-2.0)
lon =   ( -180,  180, 2.0)
# dim = value
time = '2018-05-19T12:00:00'

c_world = podpac.Coordinates([podpac.crange(*lat),
                              podpac.crange(*lon),
                              time], dims=['lat', 'lon', 'time'])
o = sm.execute(c_world)
figure()
o.plot(cmap='gist_earth_r')
axis('scaled')

## 1.1: Let's do the same thing without using PODPAC

### 1.1.0: First create an authenticated session

In [ ]:
# First create an authenticated session
import requests
class SessionWithHeaderRedirection(requests.Session):
    """
    Modified from: https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+Python
    overriding requests.Session.rebuild_auth to mantain headers when redirected
    
    Attributes
    ----------
    auth : tuple
        (username, password) string in plain text
    AUTH_HOST : str
        Host address (eg. http://example.com) that gets authenticated
    """

    AUTH_HOST = 'urs.earthdata.nasa.gov'

    def __init__(self, username=None, password=None):
        '''
        Parameters
        ------------
        username: str, optional
            Username used for authentication. 
        password: str
            Password used for authentication. 
        '''
        super(SessionWithHeaderRedirection, self).__init__()
        self.auth = (username, password)

    
    def rebuild_auth(self, prepared_request, response):
        """
        Overrides from the library to keep headers when redirected to or from
        the NASA auth host.
        """
        headers = prepared_request.headers
        url = prepared_request.url

        if 'Authorization' in headers:
            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)

            if (original_parsed.hostname != redirect_parsed.hostname) and \
                    redirect_parsed.hostname != self.AUTH_HOST and \
                    original_parsed.hostname != self.AUTH_HOST:
                del headers['Authorization']

        return

auth_session = SessionWithHeaderRedirection(username=username, password=password)

### 1.1.1: Now get the data using pydap
#### 1.1.1.0: Open the dataset and get the correct key

In [ ]:
import pydap.client
# The user needs to know/construct this url. Note the version number Vv4030_001 may change unexpectedly
source = ('https://n5eil01u.ecs.nsidc.org:443/opendap'
          '/SMAP/SPL4SMAU.004/2018.05.19'
          '/SMAP_L4_SM_aup_20180519T120000_Vv4030_001.h5')
# This line is needed for an initial authentication
auth_session.get(source + '.dds')  
dataset = pydap.client.open_url(source, session=auth_session)
print (str(dataset.keys).replace(',', '\n'))

### 1.1.1.1: Select the correct key and retrieve the data along with the correct coordinates

In [ ]:
key = 'Analysis_Data_sm_surface_analysis'
lat_key = 'cell_lat'
lon_key = 'cell_lon'
data = dataset[key][:]
# Replace no number values with nan's
data[data == -9999] = np.nan
lat = dataset[lat_key][:]
lon = dataset[lon_key][:]
print (data.shape, lat.shape, lon.shape)

### 1.1.2: Do a nearest neighbor interpolation onto the desired grid and plot

In [ ]:
import xarray as xr
grid_lat = np.linspace(90, -90, 91)
grid_lon = np.linspace(-180, 180, 181)
data_arr = xr.DataArray(data, dims=['lat', 'lon'], coords=[lat[:, 0], lon[0, :]])
data_grid = data_arr.reindex({'lat': grid_lat, 'lon': grid_lon}, method='nearest')
figure()
data_grid.plot(cmap='gist_earth_r')
axis('scaled')

# 2: Retrieve and plot SMAP data for:
## * A date range
## * With user-specified lat-lon points
## 2.0: Using PODPAC

In [ ]:
# Look at a list of points
lat_lon_pts = [
    (  45.0, 45.0,  0.0, 45.0),  # Lat
    (-100.0, 20.0, 20.0, 100.0), # Lon
]

# Turn the list into a numpy array
lat_lon_pts = np.array(lat_lon_pts)

# Let's plot the points
figure()
o.plot(cmap='gist_earth_r')
plot(lat_lon_pts[1], lat_lon_pts[0], 'ro')

In [ ]:
c_pts = podpac.Coordinates([lat_lon_pts,
                            podpac.crange('2018-05-15T00', '2018-05-19T00', '3,h')], 
                            dims=['lat_lon', 'time']
                            )
sm.threaded = True; sm.n_threads = 6
ot = sm.execute(c_pts)
sm.threaded = False

In [ ]:
figure()
plot(ot.time, ot.data.T)
legend([str(llp) for llp in lat_lon_pts.T])
locs, labels = xticks()
xticks(locs[::2])

## 2.1: Without using PODPAC?

### 2.1.0: Find the names of the source files

In [ ]:
# I'm using PODPAC to help out
_, date_source_inds = sm.source_coordinates.intersect(c_pts, return_indices=True)
date_sources = sm.sources[date_source_inds]
sources = []
for ds in date_sources:
    _, inds = ds.source_coordinates.intersect(c_pts, return_indices=True)
    sources.append(ds.sources[inds])
sources = np.concatenate(sources)
print(len(sources))
source_urls = []
for s in sources:
    source_urls.append(s.source)
    print(source_urls[-1])

### 2.1.1 Find the nearest neighbor index for lat and lon for each point

In [ ]:
dataset = pydap.client.open_url(source_urls[0], session=auth_session)
key = 'Analysis_Data_sm_surface_analysis'
lat_key = 'cell_lat'
lon_key = 'cell_lon'
lat = dataset[lat_key][:]
lon = dataset[lon_key][:]

In [ ]:
inds = []
for pt in lat_lon_pts.T:
    lat_ind = np.argmin(np.abs(lat[:, 0] - pt[0]))
    lon_ind = np.argmin(np.abs(lon[0, :] - pt[1]))
    inds.append((lat_ind, lon_ind))
inds

### 2.1.2: Loop through all the sources and retrieve the data

In [ ]:
# This takes longer than before because there is no threading
data = []
for s in source_urls:
    dataset = pydap.client.open_url(s, session=auth_session)
    date_data = [dataset[key][int(ind[0]), int(ind[1])].item() for ind in inds]
    data.append(date_data)

### 2.1.3: Plot the results

In [ ]:
figure()
plot(data)
legend([str(llp) for llp in lat_lon_pts.T])

# 3: Retrieve and plot SMAP data for:
## * A particular date 
## * Over lat-lon range with underlying, native SMAP coordinates
## 3.0: Define a lat-lon bounding box for retrieving SMAP data

In [ ]:
bbox = podpac.Coordinates([              # (start, stop, number)
                           podpac.clinspace(   49,   25, 2),  # lat
                           podpac.clinspace( -126,  -66, 2),  # lon
                          ], dims=['lat', 'lon']
)

## 3.1: Get native coordinates of the SMAP data

In [ ]:
# !!!!! This next line will take awhile first time it is run
sm_nc = sm.native_coordinates
sm_nc

In [ ]:
# Get the SMAP native coordinates in the desired bounding box
c_intersect = sm_nc.intersect(bbox)
c_intersect = c_intersect.drop('time')
c_intersect

## 3.2: Select a specific date and time

In [ ]:
c = podpac.coordinates.merge_dims([podpac.Coordinates.grid(time='2017-04-30T21:00:00'), c_intersect])
c

## 3.3: Retrieve and plot the data

In [ ]:
o = sm.execute(c)
figure()
o.plot(cmap='gist_earth_r')
axis('scaled')

# 4: Export a pipeline for re-use elsewhere

In [ ]:
print(sm.json)